In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [ ]:
path = "D:\\Anadolu_Hayat_Datathon\\"
train_df=pd.read_csv(path+"train.csv")
train_df

In [ ]:
turn_to_nation = {1: 'TC Vatandaşı', 2: 'Mavi Kart',3:'Yabancı Uyruklu'}

# Dönüşümün gerçekleştirilmesi
train_df['PP_UYRUK'] = train_df['PP_UYRUK'].replace(turn_to_nation)

In [ ]:
train_df['SORU_GELIR_CVP'] = train_df['SORU_GELIR_CVP'].str.replace(',', '.').astype(float)

In [ ]:
for col in train_df.columns:
    print(col)
    print(train_df[col].unique())

In [ ]:
nan_counts = train_df.isna().sum()
nan_counts[nan_counts > 0]

In [ ]:
train_df= train_df.drop(columns=["SON_AY_KATKI_MIKTARI","SON_AY_KATKI_ADET",
                               "SON_CEYREK_KATKI_MIKTARI","SON_CEYREK_KATKI_ADET",
                               "SON_SENE_KATKI_MIKTARI","SON_SENE_KATKI_ADET",
                                "SORU_YATIRIM_KARAKTERI_CVP","SORU_YATIRIM_KARAKTERI_RG"])

### Yeni Sütun Oluşturma

#### Bölge Sütunu Oluşturma ( Plaka kodlarına göre)

In [ ]:
regions = {
    "Marmara ": ["10", "11", "16", "17", "22", "34", "39", "41", "54", "59", "77"],
    "Ege ": ["3", "9", "20", "35", "43", "45", "48", "64"],
    "Akdeniz ": ["1", "7", "15", "31", "32", "33", "46", "80"],
    "IA ": ["6", "18", "26", "38", "40", "42", "50", "51", "58", "66", "68", "70", "71"],
    "Karadeniz ": ["5", "8", "14", "19", "28", "29", "37", "52", "53", "55", "57", "60", "61", "67", "69", "74", "78", "81"],
    "DA ": ["4", "12", "13", "23", "24", "25", "30", "36", "44", "49", "62", "65", "75", "76"],
    "GA ": ["2", "21", "27", "47", "56", "63", "72", "73", "79"]
};

def find_region(il):
    for region, plates in regions.items():
        if(np.isnan(il)):
            return None
        elif str(int(il)) in plates:
            return region
    return None  # Eğer plaka hiçbir bölge ile eşleşmezse

train_df['BOLGE'] = train_df['IL'].apply(find_region)

In [ ]:
train_df["BOLGE"].isna()

#### Yaş Grubu Sütunu Oluşturma

In [ ]:
age_groups = {
    "18-25": list(range(18, 26)),
    "26-35": list(range(26, 36)),
    "36-45": list(range(36, 46)),
    "46-55": list(range(46, 56)),
    "56-65": list(range(56, 66)),
    "65+": list(range(66, 120))  # Burada 120, yaş grubunun sonunu temsil eden bir tahmini maksimum yaş
}

def find_ageGroup(yas):
    for group, ages in age_groups.items():
        if (yas//12) <= max(ages):
            return group
    return None  # Eğer plaka hiçbir bölge ile eşleşmezse

train_df['YAS_GRUBU'] = train_df['PP_YAS'].apply(find_ageGroup)

#### Vade Tutarı İle İlgili İşlemler

In [ ]:
def findVadeOrtalama(dfs):
    vade_sutunlari = [f'VADE_TUTAR_{i}' for i in range(12)]

    # Sadece vade tutarlarını içeren sütunları al
    vade_tutarlari = dfs[vade_sutunlari]

    # Satır bazında ortalamayı al ve yeni bir sütun oluştur
    dfs['VADE_ORTALAMASI'] = vade_tutarlari.mean(axis=1)
findVadeOrtalama(train_df)#### Ortalama Vade Değişim Hızı

#### Ortalama Vade Değişim Hızı

In [ ]:
def findVadeDegisim(dfs):
    vade_sutunlari = [f'VADE_TUTAR_{i}' for i in range(12)]
    vade_degisimleri = dfs[vade_sutunlari].diff(axis=1)  # Önceki ay ile karşılaştırarak değişim miktarlarını hesapla

    # Ortalama vade tutarı değişim hızını hesapla
    dfs['ORTALAMA_VADE_TUTARI_DEGISIM_HIZI'] = vade_degisimleri.mean(axis=1)
findVadeDegisim(train_df)#### Ortalama Vade Değişim Hızı#### ÖdemeTutarı ile ilgili İşlemler

#### ÖdemeTutarı ile ilgili İşlemler

In [ ]:
def findOdemeToplamı(dfs):
    odeme_sutunlari = [f'ODEME_TUTAR_{i}' for i in range(12)]
    odeme_tutarlari = dfs[odeme_sutunlari]
    dfs['ODEME TOPLAMI'] = odeme_tutarlari.sum(axis=1)   
findOdemeToplamı(train_df)

#### Toplam Talep Sayısı Sütunu Oluşturma

In [ ]:
train_df["TOPLAM TALEP"] = train_df["BES_AYRILMA_TALEP_ADET"]+train_df["HAYAT_AYRILMA_TALEP_ADET"]+train_df["BILGI_TALEP_ADET"]+train_df["ODEMEME_TALEP_ADET"]                  

In [ ]:
train_df['Talep_Sayısı_Kategorisi'] = pd.cut(train_df['TOPLAM TALEP'], bins=[-1, 0, 1, 3, float('inf')], labels=['0', '1', '2-3', '4+'])

#### Ekonomik Durum Sütunu Oluşturma

#### Meslek Grubu Oluşturma

In [ ]:
# Savcı, hakim, ve Hakim/Savcı diye bir şey var.
# jobs = {
#     "Çalışmıyor":["Öğrenci","Ev Hanımı","Emekli (Sosyal Güvenlik Kurumu Bilinmiyor)","SSK-Sosyal Sigortalar Kurumu Emeklileri",
#                   "Bağkur Emeklileri","Emekli Sandığı Emeklileri","Emekli","İşsiz","Emekli (Vakıf,Sandık v.b.)","Çalışmıyor",
#                   "Üniversite Öğrencisi"],
#     "Emekli": ["Emekli (Sosyal Güvenlik Kurumu Bilinmiyor)","SSK-Sosyal Sigortalar Kurumu Emeklileri",
#                   "Bağkur Emeklileri","Emekli Sandığı Emeklileri","Emekli","Emekli (Vakıf,Sandık v.b.)"],
    
#     "Teknoloji":['Yazılımcı/Programcı/Analist', 'Mühendis','Teknisyen/Tekniker','Yazılımcı','Yazılım Geliştirme Uzmanı','Teknisyen'],
#     "Sağlık":['Doktor','Eczacı',"Sağlık Personeli","Hekim","Hemşire","Veteriner Hekim","Laborant","Biyolog","Ebe","Diş Hekimi"],
    
#     "Kamu":['Doktor',"Öğretmen","Polis","Memur","Akademisyen",'Kamu Çalışanı',"Hakim/Savcı","Ordu Mensubu","Subay","Astsubay",
#             "Büro Elemanı","Ofis Elemanı","Denetçi","Müfettiş/Denetçi","Emniyet Mensubu","Uzman Erbaş","Hakim","Noter","Din Görevlisi",
#             "Bürokrat","General","Uzman Jandarma","Savcı","Amiral"],
#     "Askeri":["Ordu Mensubu","Polis","Subay","Astsubay","Emniyet Mensubu","Uzman Erbaş","General","Uzman Jandarma","Amiral"],
    
#     "Özel":['Yazılımcı/Programcı/Analist', 'Mühendis','Teknisyen/Tekniker','Yazılımcı','Yazılım Geliştirme Uzmanı','Teknisyen',
#             'Muhasebe', 'İşçi', 'Serbest Meslek', 'İşyeri Sahibi/Ortağı','Esnaf', 'Satış', 'Mimar','Banka',
#             'Danışman', 'Hizmet Çalışanı','Sigorta', 'Test Uzmanı', 'Çiftçi',  'Terzi','Tacir ve Esnaf','Finans Sektörü Çalışan',
#             'İş Adamı', 'Şoför', 'Satış/Pazarlama Çalışanı','Avukat', 'Tüccar','İletişim','Pazarlama', 'Profesyonel Meslek Mensupları',
#             'Sanatçı', 'Mali Müşavir/Muhasebeci','Halkla İlişkiler', 'Medya', 'İş Analisti', 'Finans', 'Analist','Güvenlik Görevlisi',
#             'Mali Müşavir','Aşçı','Yazar', 'CEO'  'Müteahhit', 'Reklamcı/Halka İlişkiler','Sporcu', 
#             'Şoför/Makine Operatörü', 'Medya Mensubu', 'Garson', 'Üst Düzey Yönetici', 'Sivil Pilot', 'Turizmci','Radyolog', 
#             'Departman Yöneticisi', 'Reklam', 'Bakıcı', 'Komi','Zanaatkar', 'Kimyager', 'Pilot/Kaptan','Antrenör','Takım/Ekip Lideri',
#             'Emlakçı', 'Yönetim Kurulu Üyesi', 'Proje Yöneticisi', 'Barmen', 'Kuyumcu/Döviz Bürosu Çalışanı','Şair', 'Balıkçı'],
    
#     "Ekonomi":["Muhasebe","Sigorta","Banka","Finans Sektörü Çalışan","Mali Müşavir/Muhasebeci","Finans","Mali Müşavir","Kuyumcu/Döviz Bürosu Çalışanı"],
#     "Diğer":["Diğer","Bilinmiyor","nan"]
# }
# def find_jobGroup(job):
#     for group, jobss in jobs.items():
#         if job in jobss:
#             return group
#     return None  # Eğer meslek hiçbir bölge ile eşleşmezse
# train_df['SEKTOR'] = train_df['PP_MESLEK'].apply(find_jobGroup)
# train_df['SEKTOR']

### Non-Numeric Featurelar

* Label
* FLAG
* PP_Meslek
* SORU_YATIRIM_KARAKTERI_CVP
* SORU_MEDENI_HAL_CVP
* SORU_EGITIM_CVP

## Kategorik Sütun Encodingi

In [ ]:
train_df['FLAG'] = train_df['FLAG'].replace('MAYIS', 5)
train_df['FLAG'] = train_df['FLAG'].replace('HAZIRAN', 6)
train_df['FLAG'] = train_df['FLAG'].replace('TEMMUZ', 7)
train_df['FLAG'] = train_df['FLAG'].replace('AGUSTOS', 8)
train_df['FLAG'] = train_df['FLAG'].replace('EYLUL', 9)
train_df['FLAG'] = train_df['FLAG'].replace('EKIM', 10)
train_df

### Dış Veri Ekleme

#### Enflasyon, Döviz

In [ ]:
# Verileri içeren metin tablosunu oluştur
#11 61.98 3.28
data = """
10 61.36 3.43
09 61.53 4.75
08 58.94 9.09
07 47.83 9.49
06 38.21 3.92
05 39.59 0.04
"""
# StringIO kullanarak metin tablosunu oku ve DataFrame oluştur
df = pd.read_csv(StringIO(data), sep='\s+', header=None, names=['FLAG', 'Inflation', 'Value2'])
df = df.drop("Value2", axis = 1)
df = df.sort_values(by='FLAG')
df =df.reset_index()
df = df.drop("index", axis = 1)
display(df)


# 6	11	28.637082	30.902518
# 7	12	29.074857	31.714886
df2 = pd.read_excel(path+"veri.xlsx")
df2 = df2.drop(df2.index[:40])
df2 = df2.drop(df2.index[6:])
df2 = df2.drop(columns=["TP DK GBP S YTL","TP FG J0","Tarih"],axis = 1)
df2 =df2.reset_index()
df2 = df2.drop("index", axis = 1)
df2["FLAG"] = 5 + df2.index 
display(df2)

forecast_df = pd.DataFrame(index=[6, 7])
for i in ["TP DK USD S YTL","TP DK EUR S YTL",]:
    data = df2[[i,"FLAG"]]
    model = ARIMA(data[i], order=(1, 2, 1))  

    # Modeli eğitme
    fit_model = model.fit()

    # Tahmin yapma
    forecast = fit_model.forecast(steps=2)  # Gelecek 2 adım için tahmin yapılır

    forecast_df[i] = forecast
#     print("ARIMA Forecast:", forecast)

model = ARIMA(df["Inflation"], order=(1, 2, 1))  

# Modeli eğitme
fit_model = model.fit()

# Tahmin yapma
forecast = fit_model.forecast(steps=2)  
forecast_df["Inflation"] = forecast
print("ARIMA Forecast:", forecast)
display(forecast)

# Sütunu DataFrame'in başına ekleme
forecast_df.insert(0, 'FLAG', ['11','12'])
df2['Inflation'] = df['Inflation'].iloc[:]
df2 = pd.concat([df2, forecast_df])
df2

In [ ]:
train_df = pd.merge(train_df,df2,on='FLAG', how='left' )

### NaN Handling

In [ ]:
nan_counts = train_df.isna().sum()
nan_counts[nan_counts > 0]

In [ ]:
column_names_with_nan = nan_counts[nan_counts > 0].index.tolist()
for col in column_names_with_nan:
    print(col)
    if(col not in ["BOLGE","YAS_GRUBU","PP_MESLEK","Talep_Sayısı_Kategorisi","SORU_MEDENI_HAL_CVP","SORU_EGITIM_CVP"]):
        train_df[col].fillna(-train_df[col].median(), inplace=True)

In [ ]:
bins = [-101, 1000,10000,50000,100000,500000,1000000,10000000,1000000000000]

train_df['SORU_GELIR_CVP'] = pd.cut(train_df['SORU_GELIR_CVP'], bins)

train_df['SORU_GELIR_CVP'].value_counts()

In [ ]:
cols = ["SORU_MEDENI_HAL_CVP","SORU_EGITIM_CVP","PP_MESLEK"
       ,"YAS_GRUBU","BOLGE","PP_UYRUK","SORU_GELIR_CVP","Talep_Sayısı_Kategorisi"]
for col in cols  :
    one_hot_encoded = pd.get_dummies(train_df[col])
    one_hot_encoded = one_hot_encoded.astype(int)
    train_df = pd.concat([train_df, one_hot_encoded], axis=1)
    train_df = train_df.drop(col, axis=1,)

In [ ]:
rg_cols = ["SORU_EGITIM_RG","SORU_GELIR_RG","SORU_COCUK_SAYISI_RG","SORU_MEDENI_HAL_RG","SORU_EGITIM_RG"]
for col in rg_cols:
    train_df[col]+= 11-train_df["FLAG"]
train_df = train_df.drop("FLAG", axis =1)

In [ ]:
train_df.columns = train_df.columns.astype(str)

In [ ]:
train_df.columns.tolist()

### Test Set

In [ ]:
test_df = pd.read_csv(path+"test.csv")
test_df.head()

In [ ]:
test_df['PP_UYRUK'] = test_df['PP_UYRUK'].replace(turn_to_nation)

In [ ]:
test_df['SORU_GELIR_CVP'] = test_df['SORU_GELIR_CVP'].str.replace(',', '.').astype(float)

In [ ]:
nan_counts = test_df.isna().sum()
nan_counts[nan_counts > 0]

In [ ]:
test_df['BOLGE'] = test_df['IL'].apply(find_region)
test_df['YAS_GRUBU'] = test_df['PP_YAS'].apply(find_ageGroup)
findVadeOrtalama(test_df)#### Ortalama Vade Değişim Hızı
findVadeDegisim(test_df)    
findOdemeToplamı(test_df)
test_df["TOPLAM TALEP"] = test_df["BES_AYRILMA_TALEP_ADET"]+test_df["HAYAT_AYRILMA_TALEP_ADET"]+test_df["BILGI_TALEP_ADET"]+test_df["ODEMEME_TALEP_ADET"]                  
test_df['Talep_Sayısı_Kategorisi'] = pd.cut(test_df['TOPLAM TALEP'], bins=[-1, 0, 1, 3, float('inf')], labels=['0', '1', '2-3', '4+'])

In [ ]:
test_df['FLAG'] = test_df['FLAG'].replace('KASIM', 11)
test_df['FLAG'] = test_df['FLAG'].replace('ARALIK', 12)

In [ ]:
df3 = df2[6:]
df3['FLAG'] = df3['FLAG'].astype('int64')
df3

In [ ]:
test_df = pd.merge(test_df,df3, on='FLAG', how='left' )
test_df

In [ ]:
test_df= test_df.drop(columns=["SON_AY_KATKI_MIKTARI","SON_AY_KATKI_ADET",
                               "SON_CEYREK_KATKI_MIKTARI","SON_CEYREK_KATKI_ADET",
                               "SON_SENE_KATKI_MIKTARI","SON_SENE_KATKI_ADET",
                               "SORU_YATIRIM_KARAKTERI_CVP","SORU_YATIRIM_KARAKTERI_RG","FLAG"])

In [ ]:
nan_counts = test_df.isna().sum()
nan_counts[nan_counts > 0]

In [ ]:
column_names_with_nan = nan_counts[nan_counts > 0].index.tolist()
for col in column_names_with_nan:
    if(col not in ["BOLGE","YAS_GRUBU","PP_MESLEK","Talep_Sayısı_Kategorisi","SORU_MEDENI_HAL_CVP","SORU_EGITIM_CVP"]):
        test_df[col].fillna(-test_df[col].median(), inplace=True)

In [ ]:
test_df = test_df.drop('MUSTERI_ID',axis = 1)
test_df['Marriage Annulled'] = 0

In [ ]:
bins = [-101, 1000,10000,50000,100000,500000,1000000,10000000,1000000000000]

test_df['SORU_GELIR_CVP'] = pd.cut(test_df['SORU_GELIR_CVP'], bins)
test_df['SORU_GELIR_CVP'].value_counts()

In [ ]:
cols = ["SORU_MEDENI_HAL_CVP","SORU_EGITIM_CVP","PP_MESLEK"
       ,"YAS_GRUBU","BOLGE","PP_UYRUK","SORU_GELIR_CVP","Talep_Sayısı_Kategorisi"]
for col in cols  :
    one_hot_encoded = pd.get_dummies(test_df[col])
    one_hot_encoded = one_hot_encoded.astype(int)
    test_df = pd.concat([test_df, one_hot_encoded], axis=1)
    test_df = test_df.drop(col, axis=1,)

In [ ]:
test_df.columns = test_df.columns.astype(str)

In [ ]:
test_df.isna().sum().tolist()

## Modelling

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn.over_sampling import SMOTE

In [ ]:
UA_samples = train_df[(train_df['LABEL'] == 'UA') ].sample(n=130000, random_state=42)

# Diğer sınıflardaki örnekler
other_samples = train_df[train_df['LABEL'] != 'UA']

# Yeni veri setini birleştirelim
balanced_data = pd.concat([UA_samples, other_samples])

X = balanced_data.drop(columns = ['LABEL','MUSTERI_ID'], axis=1)
y = balanced_data['LABEL']

unique_values, counts = np.unique(y, return_counts=True)

# Sonuçları ekrana yazdıralım
for value, count in zip(unique_values, counts):
    print(f"{value}: {count} adet")

In [ ]:
# SMOTE'u kullanarak synthetic data generation yapalım
smote = SMOTE(sampling_strategy={'HU06':5822 , 'HU07':6272, 'HU11': 3541, 'HU12': 3824, 'HU14': 10754, 'HU15': 3585, 'HU19': 4500, 'UA': 130000},k_neighbors=3, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
synthetic_data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.DataFrame(y_resampled, columns=['LABEL'])], axis=1)

In [ ]:
synthetic_data

In [ ]:
# Veri setini ayırma
X = synthetic_data.drop(columns = ['LABEL'], axis=1)  # Bağımsız değişkenler
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(synthetic_data['LABEL'])  # Bağımlı değişken
# y = synthetic_data['LABEL'] # Bağımlı değişken
# test_df = test_df[X.columns]
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
# RandomForestClassifier modelini tanımlama ve eğitme
# class_weights = {'HU06': 5, 'HU07':51, 'HU11': 10, 'HU12': 10, 'HU14': 1, 'HU15': 10, 'HU19': 10, 'UA': 0.00001}

In [ ]:
unique_values, counts = np.unique(y, return_counts=True)
for value, count in zip(unique_values, counts):
    print(f"{value}: {count} adet")

In [ ]:
X_egitim, X_test, y_egitim, y_test = train_test_split(X, y, test_size=0.1, random_state=42,stratify=y)

In [ ]:
def custom_f1_score(y_true, y_pred):
    # F1 skorunu hesapla
    f1 = f1_score(y_true, y_pred, average=None)
    
    # Sınıf katsayıları
    class_coefficients = {
        'HU06': 0.0385,
        'HU07': 0.0328,
        'HU11': 0.2791,
        'HU12': 0.1812,
        'HU14': 0.0113,
        'HU15': 0.2952,
        'HU19': 0.1614,
        'UA': 0.0001
    }
    
    # Sınıf bazında F1 skorlarını katsayılarla çarp
    weighted_f1 = [f1[i] * class_coefficients[class_] for i, class_ in enumerate(class_coefficients.keys())]
    
    # Toplam ağırlıklı F1 skorunu döndür
    return sum(weighted_f1)

In [ ]:
xgb_model = XGBClassifier()
cat_model = CatBoostClassifier()
param_grid = {
    'max_depth': [5,7,9,],
    'learning_rate': [0.1, 0.2,0.3],
}

custom_scorer = make_scorer(custom_f1_score)

# GridSearchCV'yi tanımlayın ve özel skorlama fonksiyonunu kullanın
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring=custom_scorer, cv=3)

# GridSearchCV'yi uygulayın
grid_search.fit(X, y)

In [ ]:
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

In [ ]:
model = CatBoostClassifier(learning_rate=0.25, max_depth=7,iterations=1000)
model.fit(X_egitim, y_egitim)

# Test seti üzerinde tahmin yapma
# test_df = scaler.fit_transform(test_df)
tahminler = model.predict(X_test)

In [ ]:
print(custom_f1_score(tahminler,y_test))

In [ ]:
model = CatBoostClassifier(learning_rate=0.25, max_depth=7,n_estimators =1000)
model.fit(X, y,verbose=True)

# Test seti üzerinde tahmin yapma
test_df = scaler.fit_transform(test_df)
tahminler = model.predict(test_df)

In [ ]:
kategori_numerik_dict = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Sözlüğü ekrana yazdırma
print("Kategori -> Numerik Değer:")
for kategori, numerik_değer in kategori_numerik_dict.items():
    print(f"{kategori} -> {numerik_değer}")

In [ ]:
unique_values, counts = np.unique(y, return_counts=True)

# Sonuçları ekrana yazdıralım
for value, count in zip(unique_values, counts):
    print(f"{value}: {count} adet")

In [ ]:
result_df = pd.read_csv(path+"sample_submission.csv")
result_df["LABEL"] = tahminler

label_dict = {
    0: "HU06",
    1: "HU07",
    2: "HU11",
    3: "HU12",
    4: "HU14",
    5: "HU15",
    6: "HU19",
    7: "UA"
}

# "LABEL" sütununu değiştirme
result_df["LABEL"] = result_df["LABEL"].map(label_dict)

result_df.to_csv('C:\\Users\\ACER\\Desktop\\submission.csv', index=False)